# Feature Engineering (Part II)
This notebook will be used to build the function that will calculate all of the quarterly percentage changes and rate of change features. It will add the data back to the file. Consider calcualtions for percent of revenue.


In [1]:
# File system libraries
import os
from google.colab import drive

# Data Manipulation Libraries
import numpy as np
import pandas as pd

# Stat Libraries
import scipy.stats as stats

# Machine Learning Libraries
#import pycaret #Not working with this version of python
import sklearn

# Visualization Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt


In [2]:
# show decimals without scientific notation
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
# Mount the google drive
drive.mount('/content/drive')
# Navigate to the folder and set the file name
path = '/content/drive/MyDrive/Colab Notebooks/696 - Milestone II/696 - Milestone II - Shared/Dataset'

os.chdir(path)
os.getcwd()
os.listdir()

Mounted at /content/drive


['Russell_3000.csv',
 'Russell_1000',
 'Macroeconomics',
 'Russel_3000_09-18_merged_cleaned_KPIs_QoQ.csv',
 'Russell_3000_Fundamentals_Macroeconomics_KPI_0919.csv',
 'older',
 'Russell 3000 Data column summary.gdoc']

In [7]:
filename = 'Russell_3000_Fundamentals_Macroeconomics_KPI_0919.csv'
dataset = pd.read_csv(filename)
print(dataset.head())

  Ticker                        Name                  Sector Exchange  \
0   NVDA                 NVIDIA CORP  Information Technology   NASDAQ   
1   MSFT              MICROSOFT CORP  Information Technology   NASDAQ   
2   AAPL                   APPLE INC  Information Technology   NASDAQ   
3   AMZN              AMAZON COM INC  Consumer Discretionary   NASDAQ   
4   META  META PLATFORMS INC CLASS A           Communication   NASDAQ   

        Location      Market Value   Price      Quantity  Weight (%)  \
0  United States  1,066,994,615.04  171.66  6,215,744.00        6.39   
1  United States  1,000,536,825.69  507.97  1,969,677.00        5.99   
2  United States    938,729,828.14  239.78  3,914,963.00        5.62   
3  United States    600,426,381.12  235.68  2,547,634.00        3.60   
4  United States    434,118,178.20  748.65    579,868.00        2.60   

   CashAndSTInvestments_2024Q2  ...  IndustrialProd 2024Q2  \
0                          NaN  ...                 102.71   
1   

In [ ]:
dataset.shape

(2570, 122)

Alright, now that we have it all loaded, we are going to have to look at quarterly changes. From the quarterly changes we can start calculating the rate of change etc. The best way to do this I think will be to design a function that will output a series that we can then just add into the dataframe. As parameters, it will take our original dataframe, the fature and the quarters to calculate the change. First, let's get a list of all the columns that we will need to calculate this for.

In [8]:
# Let's create a copy of the dataset so that we can merge data back to it later
original_df = dataset.copy()
columns = dataset.columns
for column in columns:
    print(column)

Ticker
Name
Sector
Exchange
Location
Market Value
Price
Quantity
Weight (%)
CashAndSTInvestments_2024Q2
CashAndSTInvestments_2024Q3
CashAndSTInvestments_2024Q4
CashAndSTInvestments_2025Q1
CashAndSTInvestments_2025Q2
CashFromOps_2024Q2
CashFromOps_2024Q3
CashFromOps_2024Q4
CashFromOps_2025Q1
CashFromOps_2025Q2
EPS_2024Q2
EPS_2024Q3
EPS_2024Q4
EPS_2025Q1
EPS_2025Q2
NetIncome_2024Q2
NetIncome_2024Q3
NetIncome_2024Q4
NetIncome_2025Q1
NetIncome_2025Q2
OperatingIncome_2024Q2
OperatingIncome_2024Q3
OperatingIncome_2024Q4
OperatingIncome_2025Q1
OperatingIncome_2025Q2
Revenue_2024Q2
Revenue_2024Q3
Revenue_2024Q4
Revenue_2025Q1
Revenue_2025Q2
LongTermDebt_2024Q2
LongTermDebt_2024Q3
LongTermDebt_2024Q4
LongTermDebt_2025Q1
LongTermDebt_2025Q2
ShortTermDebtOrCurrentLiab_2024Q2
ShortTermDebtOrCurrentLiab_2024Q3
ShortTermDebtOrCurrentLiab_2024Q4
ShortTermDebtOrCurrentLiab_2025Q1
ShortTermDebtOrCurrentLiab_2025Q2
TotalAssets_2024Q2
TotalAssets_2024Q3
TotalAssets_2024Q4
TotalAssets_2025Q1
TotalAssets_2

Okay, so the easy part is the quarters so let's build a list of quarters

In [9]:
quarters = ['_2024Q2','_2024Q3','_2024Q4','_2025Q1','_2025Q2']

Now, we need to create a list of unique columns. We can do this by iterating trhough splitting adding to a set and then creating a list.

In [11]:
unique_columns = set()
for column in columns:
    column = column.replace(' ','_')
    words = column.split('_')
    if words[0] != 'KPI':
        unique_columns.add(words[0])
    else:
        unique_columns.add(str(words[0]) + '_' + str(words[1]))
unique_columns = list(unique_columns)
for column in unique_columns:
    print(column)


CurrentLiabilities
Market
Weight
CashAndSTInvestments
KPI_Leverage
KPI_ReturnOnAssets
Inflation
LongTermDebt
ShortTermDebtOrCurrentLiab
Unemployment
IncomeTaxExpense
KPI_DebtToEquityRatio
Location
Price
Name
IndustrialProd
GDP
TotalAssets
TotalDebt
OperatingIncome
Sector
OtherOperatingExpense
KPI_TotalAssetTurnover
Revenue
CapitalExpenditure
CashFromOps
Exchange
Ticker
KPI_GrossProfitMargin
CostOfRevenue
EPS
TotalEquity
KPI_NetProfitMargin
NetIncome
TotalLiabilities
KPI_ReturnOnEquity
Quantity
InterestRate
InterestExpense
CurrentAssets


Great, now we can drop all of the features that are not quarterly as we won't be calculating the differences for these.

In [30]:
drop = {'Notional','Name','Ticker','Exchange','Price','Quantity','Sector','Market','Location','Weight','Inflation','Unemployment','IndustrialProd','GDP','InterestRate'}

unique_columns = [c for c in unique_columns if c not in drop]
for c in unique_columns:
    print(c)

CurrentLiabilities
CashAndSTInvestments
KPI_Leverage
KPI_ReturnOnAssets
LongTermDebt
ShortTermDebtOrCurrentLiab
IncomeTaxExpense
KPI_DebtToEquityRatio
TotalAssets
TotalDebt
OperatingIncome
OtherOperatingExpense
KPI_TotalAssetTurnover
Revenue
CapitalExpenditure
CashFromOps
KPI_GrossProfitMargin
CostOfRevenue
EPS
TotalEquity
KPI_NetProfitMargin
NetIncome
TotalLiabilities
KPI_ReturnOnEquity
InterestExpense
CurrentAssets


Okay, Now we have a solid list of the raw data that we want to calcualte quarterly information for, we can start to build the function that will accomplish what we want it to. We can return a series, or we can just have the function build it into the dataset without returning anything, which is likely way more efficient. Thinking about the nesting of this, we will iterate through each unique column, we will then iterate through the quarters to calcuate the values and add it to the dataframe before moving on to the next unique column.

In [31]:
def quarterly_changes(dataset, unique_columns, quarters):
    for column in unique_columns:
        try:
            for val in range(1,len(quarters)):
                series_1 = dataset[str(column) + str(quarters[val-1])]
                series_2 = dataset[str(column) + str(quarters[val])]
                dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1
        except:
            for val in range(2,len(quarters)):
                series_1 = dataset[str(column) + str(quarters[val-1])]
                series_2 = dataset[str(column) + str(quarters[val])]
                dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1

    print(f"Completed Quarterly Change Calculations")
    return dataset


In [32]:
# Run the function to see the output
dataset = quarterly_changes(dataset, unique_columns, quarters)
print(dataset.shape)
dataset.head()

Completed Quarterly Change Calculations
(2570, 267)


/tmp/ipython-input-3750132376.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1
/tmp/ipython-input-3750132376.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dataset[f'{column}_QoQ_{quarters[val-1][-4:]}_{quarters[val][-4:]}'] = (series_2 - series_1)/series_1
/tmp/ipython-input-3750132376.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

,Ticker,Name,Sector,Exchange,Location,Market Value,Price,Quantity,Weight (%),CashAndSTInvestments_2024Q2,...,KPI_ReturnOnEquity_QoQ_24Q4_25Q1,KPI_ReturnOnEquity_QoQ_25Q1_25Q2,InterestExpense_QoQ_24Q2_24Q3,InterestExpense_QoQ_24Q3_24Q4,InterestExpense_QoQ_24Q4_25Q1,InterestExpense_QoQ_25Q1_25Q2,CurrentAssets_QoQ_24Q2_24Q3,CurrentAssets_QoQ_24Q3_24Q4,CurrentAssets_QoQ_24Q4_25Q1,CurrentAssets_QoQ_25Q1_25Q2
0,NVDA,NVIDIA CORP,Information Technology,NASDAQ,United States,"1,066,994,615.04",171.66,"6,215,744.00",6.39,NaN,...,-0.02,-0.24,NaN,0.00,0.00,0.03,NaN,0.13,0.18,0.12
1,MSFT,MICROSOFT CORP,Information Technology,NASDAQ,United States,"1,000,536,825.69",507.97,"1,969,677.00",5.99,"18,315,000,000.00",...,0.01,-0.01,-0.17,0.02,0.00,0.04,-0.06,-0.02,0.07,0.22
2,AAPL,APPLE INC,Information Technology,NASDAQ,United States,"938,729,828.14",239.78,"3,914,963.00",5.62,"25,565,000,000.00",...,-0.37,-0.05,-0.31,1.47,-0.32,-0.05,0.22,-0.13,-0.11,0.03
3,AMZN,AMAZON COM INC,Consumer Discretionary,NASDAQ,United States,"600,426,381.12",235.68,"2,547,634.00",3.60,"71,178,000,000.00",...,-0.21,-0.02,0.02,-0.05,-0.05,-0.05,0.01,0.09,-0.03,0.04
4,META,META PLATFORMS INC CLASS A,Communication,NASDAQ,United States,"434,118,178.20",748.65,"579,868.00",2.60,"32,045,000,000.00",...,-0.25,0.07,0.62,0.21,-0.04,0.00,0.19,0.10,-0.10,-0.18


Let's now take all of these rates of change, plot them and take the line of best fit so that we can get the slope as an accurate rate of change over the past 5 quarters.

In [33]:
# Let's get all of the columns again
columns = dataset.columns
# Use sets to avoid duplicates
QoQ_columns = set()
QoQ_quarters = set()
for column in columns:
    if 'QoQ' in column:
        # Pull out the common Column Name
        QoQ_columns.add(column[:-10])
        # Let's also pull out the QoQ
        QoQ_quarters.add(column[-10:])
QoQ_columns = list(QoQ_columns)
QoQ_quarters = list(QoQ_quarters)
# Order will be important so let's sort them
QoQ_quarters.sort()
QoQ_quarters


['_24Q2_24Q3', '_24Q3_24Q4', '_24Q4_25Q1', '_25Q1_25Q2']

Now that we have a list of all of the features, we need to start pulling out all of the data in a funciton.

In [34]:
def get_QoQ_rate_data(row, QoQ_columns, QoQ_quarters):
    for column in QoQ_columns:
        values = []
        try:
            for quarter in QoQ_quarters[:-1]:
                values.append(row[column + quarter])
        except:
            # Addressing variables that don't have intial Quarter
            for quarter in QoQ_quarters[1:-1]:
                values.append(row[column + quarter])
        pd.Series(values).dropna(inplace=True)
        y = np.array(values,float)
        t = np.arange(len(y))
        # Get the OLS slope
        b = np.polyfit(t,y,1)[0]
        row[column + '_Rate'] = b
    return row

def get_raw_rate_data(row, unique_columns, quarters):
    for column in unique_columns:
        values = []
        try:
            for quarter in quarters[:-1]:
                values.append(row[column + quarter])
        except:
            # Addressing variables that don't have intial Quarter
            for quarter in quarters[1:-1]:
                values.append(row[column + quarter])
        pd.Series(values).dropna(inplace=True)
        y = np.array(values,float)
        t = np.arange(len(y))
        # Get the OLS slope
        b = np.polyfit(t,y,1)[0]
        row[column + '_Rate'] = b
    return row

Alright now that we have the function let's apply it to our dataset.

In [35]:
dataset = dataset.apply(lambda row: get_QoQ_rate_data(row, QoQ_columns, QoQ_quarters), axis=1)
dataset = dataset.apply(lambda row: get_raw_rate_data(row, unique_columns, quarters), axis=1)
dataset.head()

,Ticker,Name,Sector,Exchange,Location,Market Value,Price,Quantity,Weight (%),CashAndSTInvestments_2024Q2,...,KPI_GrossProfitMargin_Rate,CostOfRevenue_Rate,EPS_Rate,TotalEquity_Rate,KPI_NetProfitMargin_Rate,NetIncome_Rate,TotalLiabilities_Rate,KPI_ReturnOnEquity_Rate,InterestExpense_Rate,CurrentAssets_Rate
0,NVDA,NVIDIA CORP,Information Technology,NASDAQ,United States,"1,066,994,615.04",171.66,"6,215,744.00",6.39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MSFT,MICROSOFT CORP,Information Technology,NASDAQ,United States,"1,000,536,825.69",507.97,"1,969,677.00",5.99,"18,315,000,000.00",...,-0.00,"840,500,000.00",0.15,"17,521,400,000.00",0.01,"1,080,500,000.00","-1,294,600,000.00",-0.00,"-30,900,000.00","-1,211,600,000.00"
2,AAPL,APPLE INC,Information Technology,NASDAQ,United States,"938,729,828.14",239.78,"3,914,963.00",5.62,"25,565,000,000.00",...,0.00,"2,815,300,000.00",0.22,"1,007,200,000.00",0.02,"3,159,000,000.00","-3,210,400,000.00",0.07,"3,159,000,000.00","-4,003,000,000.00"
3,AMZN,AMAZON COM INC,Consumer Discretionary,NASDAQ,United States,"600,426,381.12",235.68,"2,547,634.00",3.60,"71,178,000,000.00",...,-0.00,"2,748,900,000.00",0.14,"23,507,900,000.00",0.01,"1,560,200,000.00","7,050,300,000.00",-0.00,"-17,700,000.00","4,908,900,000.00"
4,META,META PLATFORMS INC CLASS A,Communication,NASDAQ,United States,"434,118,178.20",748.65,"579,868.00",2.60,"32,045,000,000.00",...,0.00,"225,600,000.00",0.58,"10,290,600,000.00",0.02,"1,468,700,000.00","6,666,500,000.00",-0.00,"37,900,000.00","5,036,600,000.00"


In [36]:
missing = dataset['CapitalExpenditure_QoQ_25Q1_25Q2'].isna().sum()
total = dataset.shape[0]
ratio =(missing/total)*100
print(f'The percent of missing data in the rates is {np.round(ratio,2)}%')

The percent of missing data in the rates is 5.99%


In [38]:
print(dataset.KPI_GrossProfitMargin_Rate.min())
print(dataset.KPI_GrossProfitMargin_Rate.max())

-1.7846938672451476
94.65243828065641


## Final Section to Save new CSV

In [37]:
# Uncomment this block when we want to save to a CSV file
dataset.to_csv("Russel_3000_09-18_merged_cleaned_KPIs_QoQ.csv", index=False)